In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import glob
import time, os, sys

import numpy as np
import pandas as pd

from PIL import Image
import tifffile

from cellpose_omni import models, core, utils, transforms, plot
import ncolor

### Specify inputs

In [4]:
paths = ['241208_160806_jg_ve_norha_mw_encapsulation_dapi_phal_d7_40k_60k/241208_160806_Plate 1/#24B2570319_ZProj',
         '241208_174217_jg_ve_norha_mw_encapsulation_dapi_phal_d7_peg_10k/241208_174217_Plate 1/#24B2570319_ZProj',
         '241125_192553_jg_ve_norha_mw_encapsulation_dapi_phal_d3_peg_10k/241125_193216_Plate 1/#24B2570319_ZProj',
         '241126_091801_jg_ve_norha_mw_encapsulation_dapi_phal_d3_40k_60k/241126_091801_Plate 1/#24B2570319_ZProj',
         '241208_124427_jg_ve_norha_mw_encapsulation_dapi_phal_d14_peg_10k/241208_124736_Plate 1/#24B2570319_ZProj',
         '241208_144351_jg_ve_norha_mw_encapsulation_dapi_phal_d14_40k_60k/241208_144351_Plate 1/#24B2570319_ZProj'
]
paths = ['/mnt/z/James/norbornene homopolymerization/same_formulations/' + p for p in paths]
files = [glob.glob(path+'/*001.tif') for path in paths]
files = np.array([f for sublist in files for f in sublist])

In [5]:
# Used these files to analyze cells in gels with same mechanics

#paths = ['241218_143216_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_40k_60k/241218_143532_Plate 1/#24B2570319_ZProj',
#         '241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj',
#         '241221_122140_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d7_peg_10k/241221_122140_Plate 1/#24B2570319_ZProj',
#         '241221_133248_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d7_40k_60k/241221_133248_Plate 1/#24B2570319_ZProj',
#         '241229_114731_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d14/241229_114731_Plate 1/#24B2570319_ZProj'
#]
#paths = ['/mnt/z/James/norbornene homopolymerization/same_mechanics/' + p for p in paths]
#files = [glob.glob(path+'/*001.tif') for path in paths]
#files = np.array([f for sublist in files for f in sublist])

In [6]:
files[:1]

array(['/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_143216_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_40k_60k/241218_143532_Plate 1/#24B2570319_ZProj/C4ROI4_-2_1_1_ZProj[Confocal DAPI 405,442]_001.tif'],
      dtype='<U225')

In [7]:
nucleus = files[np.array(['DAPI' in f for f in files])]
nucleus.sort()
actin = files[np.array(['TRITC' in f for f in files])]
actin.sort()

In [8]:
nucleus[:3]

array(['/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj/A1ROI1_-2_1_1_ZProj[Confocal DAPI 405,442]_001.tif',
       '/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj/A1ROI2_-2_1_1_ZProj[Confocal DAPI 405,442]_001.tif',
       '/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj/A1ROI3_-2_1_1_ZProj[Confocal DAPI 405,442]_001.tif'],
      dtype='<U225')

In [9]:
actin[:3]

array(['/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj/A1ROI1_-2_2_1_ZProj[Confocal TRITC 556,600]_001.tif',
       '/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj/A1ROI2_-2_2_1_ZProj[Confocal TRITC 556,600]_001.tif',
       '/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj/A1ROI3_-2_2_1_ZProj[Confocal TRITC 556,600]_001.tif'],
      dtype='<U225')

### Specify cellpose arguments

In [11]:
# Import dependencies
import numpy as np
from cellpose_omni import models, core, io

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? {}'.format(use_GPU))

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

2024-12-29 14:01:16,254	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
>>> GPU activated? True


In [12]:
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES
import omnipose

MODEL_NAMES

['bact_phase_omni',
 'bact_fluor_omni',
 'worm_omni',
 'worm_bact_omni',
 'worm_high_res_omni',
 'cyto2_omni',
 'plant_omni',
 'bact_phase_cp',
 'bact_fluor_cp',
 'plant_cp',
 'worm_cp',
 'cyto',
 'nuclei',
 'cyto2']

In [13]:
model_name = 'cyto2'
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

2024-12-29 14:01:16,447	[INFO ]	[models.py  427  __init__          ]	>>cyto2<< model set to be used
2024-12-29 14:01:16,449	[INFO ]	[core.py    74   _use_gpu_torch    ]	** TORCH GPU version installed and working. **
2024-12-29 14:01:16,450	[INFO ]	[core.py    85   assign_device     ]	>>>> using GPU


/home/jg9zk/miniconda3/envs/omnipose/lib/python3.9/site-packages/cellpose_omni/resnet_torch.py:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

In [14]:
# define parameters
# peg & 10k gels are much less autofluorescent than 40k & 60k gels, requiring different parameters for accurate segmentation
# used these for all groups @ d3,d7,d14 10k and peg groups; nuc image to power of 1
nucleus_params = {'channels': [0,0], # always define this with the model
          'rescale': .5, # upscale or downscale your images, None = no rescaling 
          'mask_threshold': 1, # erode or dilate masks with higher or lower values 
          'flow_threshold': .0, # default is .4, but only needed if there are spurious masks to clean up; slows down output
          'transparency': True, # transparency in flow output
          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
          'cluster': False, # use DBSCAN clustering
          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
          #'verbose': False, # turn on if you want to see more output 
          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 
          'affinity_seg': False, # new feature, stay tuned...
          'diameter': 10
         }

In [15]:
# define parameters
# peg & 10k gels are much less autofluorescent than 40k & 60k gels, requiring different parameters for accurate segmentation
# used these for all groups @ d3,d7,d14 40k and 60k groups; nuc image to power of 0.5
nucleus_params = {'channels': [0,0], # always define this with the model
          'rescale': .5, # upscale or downscale your images, None = no rescaling 
          'mask_threshold': -2, # erode or dilate masks with higher or lower values 
          'flow_threshold': -2, # default is .4, but only needed if there are spurious masks to clean up; slows down output
          'transparency': True, # transparency in flow output
          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
          'cluster': False, # use DBSCAN clustering
          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
          #'verbose': False, # turn on if you want to see more output 
          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 
          'affinity_seg': False, # new feature, stay tuned...
          'diameter': 5
         }

In [16]:
# define parameters
# used these for all groups @ d3,d7,d14 40k and 60k groups; actin image to power of 0.5
# used these for all groups @ d3,d7,d14 10k and peg groups; actin image to power of 1
actin_params = {'channels': [0,1], # always define this with the model
          'rescale': .5, # upscale or downscale your images, None = no rescaling 
          'mask_threshold': -2, # erode or dilate masks with higher or lower values 
          'flow_threshold': 0, # default is .4, but only needed if there are spurious masks to clean up; slows down output
          'transparency': True, # transparency in flow output
          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
          'cluster': True, # use DBSCAN clustering
          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
          #'verbose': False, # turn on if you want to see more output 
          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 
          'affinity_seg': False, # new feature, stay tuned...
          'diameter': 40
         }

### Run cellpose and save masks to images

In [23]:
for i,(n,a) in enumerate(zip(nucleus,actin)):
    print([i,n.split('ZProj/')[1].split('_')[0],a.split('ZProj/')[1].split('_')[0]])
    
    stackedp = np.stack([np.array(tifffile.imread(n)),np.array(tifffile.imread(a))])
    stackedp = np.stack([stackedp[0]/np.max(stackedp[0]),stackedp[1]/np.max(stackedp[1])])
    stackedp = np.stack([np.clip(stackedp[0]-np.mean(stackedp[0]),0,None)**1,
                         np.clip(stackedp[1]-0*np.median(stackedp[1]),0,None)**.5,
                         np.zeros(stackedp.shape[1:])])

    #nucleus segmentation
    masks, flows, styles = model.eval(stackedp[0],**nucleus_params);
    
    if np.sum(masks>0) > 0 :
        if np.max(masks) > 1 :
            np.save(re.sub('\.tif','.npy',n), masks)
            tifffile.imwrite(re.sub('\.tif','_masks.tif',n), masks)
            plt.imsave(re.sub('\.tif','_masks_4color.png',n), ncolor.label(masks))

    #actin segmentation
    masks, flows, styles = model.eval(stackedp,**actin_params);

    if np.sum(masks>0) > 0 :
        if np.max(masks) > 1 :
            np.save(re.sub('\.tif','.npy',a), masks)
            tifffile.imwrite(re.sub('\.tif','_masks.tif',a), masks)
            plt.imsave(re.sub('\.tif','_masks_4color.png',a), ncolor.label(masks))

[0, 'A2ROI3', 'A2ROI3']


/home/jg9zk/miniconda3/envs/omnipose/lib/python3.9/site-packages/cellpose_omni/resnet_torch.py:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

[1, 'A2ROI4', 'A2ROI4']


/home/jg9zk/miniconda3/envs/omnipose/lib/python3.9/site-packages/cellpose_omni/resnet_torch.py:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

[2, 'A2ROI5', 'A2ROI5']


...

/home/jg9zk/miniconda3/envs/omnipose/lib/python3.9/site-packages/cellpose_omni/resnet_torch.py:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

[71, 'D4ROI4', 'D4ROI4']


/home/jg9zk/miniconda3/envs/omnipose/lib/python3.9/site-packages/cellpose_omni/resnet_torch.py:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

[72, 'D4ROI5', 'D4ROI5']


/home/jg9zk/miniconda3/envs/omnipose/lib/python3.9/site-packages/cellpose_omni/resnet_torch.py:295: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

## Shape metrics of cells in hydrogels with same formulations (corrected masks)

In [18]:
from skimage import measure
from PIL import Image
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
paths = ['241208_160806_jg_ve_norha_mw_encapsulation_dapi_phal_d7_40k_60k/241208_160806_Plate 1/#24B2570319_ZProj',
         '241208_174217_jg_ve_norha_mw_encapsulation_dapi_phal_d7_peg_10k/241208_174217_Plate 1/#24B2570319_ZProj',
         '241125_192553_jg_ve_norha_mw_encapsulation_dapi_phal_d3_peg_10k/241125_193216_Plate 1/#24B2570319_ZProj',
         '241126_091801_jg_ve_norha_mw_encapsulation_dapi_phal_d3_40k_60k/241126_091801_Plate 1/#24B2570319_ZProj',
         '241208_124427_jg_ve_norha_mw_encapsulation_dapi_phal_d14_peg_10k/241208_124736_Plate 1/#24B2570319_ZProj',
         '241208_144351_jg_ve_norha_mw_encapsulation_dapi_phal_d14_40k_60k/241208_144351_Plate 1/#24B2570319_ZProj'
]
paths = ['/mnt/z/James/norbornene homopolymerization/same_formulations/' + p for p in paths]
files = [glob.glob(path+'/*TRITC 556,600]_001_masks_corrected.png') for path in paths]
files = np.array([f for sublist in files for f in sublist])
len(files)

236

In [8]:
files[0]

np.str_('/mnt/z/James/norbornene homopolymerization/same_formulations/241208_160806_jg_ve_norha_mw_encapsulation_dapi_phal_d7_40k_60k/241208_160806_Plate 1/#24B2570319_ZProj/C4ROI5_-2_2_1_ZProj[Confocal TRITC 556,600]_001_masks_corrected.png')

In [9]:
'_'.join(files[0].split('_')[:16])

'/mnt/z/James/norbornene homopolymerization/same_formulations/241208_160806_jg_ve_norha_mw_encapsulation_dapi_phal_d7_40k_60k/241208_160806_Plate 1/#24B2570319_ZProj/C4ROI5'

In [28]:
for i,f in enumerate(files):
    #read in masks for calculations
    fov = '_'.join(f.split('_')[:16])
    well, roi = fov.split('/')[-1].split('ROI')
 
    cy5_masks = measure.label(np.array(Image.open(f))[:,:,0])-1
    #cytoskeleton morphometrics
    cy5_props = pd.DataFrame(measure.regionprops_table(cy5_masks*1, properties=['axis_major_length','axis_minor_length','perimeter',
                                                                              'area','eccentricity']))
    cy5_props['labels'] = np.arange(1,len(cy5_props)+1)
    cy5_props['aspect'] = cy5_props['axis_major_length']/cy5_props['axis_minor_length']
    cy5_props['form_factor'] = 4*(np.pi)*cy5_props['area']/(cy5_props['perimeter']**2)

    cy5_props['file'] = f
    cy5_props['fov'] = fov
    cy5_props['well'] = well
    cy5_props['roi'] = roi
    mhc_list.append(cy5_props)
    
    if i % 10 == 0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [29]:
mhc_list = pd.concat(mhc_list)

In [30]:
mhc_list.to_csv('/mnt/z/James/norbornene homopolymerization/same_formulations/cyto_metrics_no_filtering.csv')

## Shape metrics of cells in hydrogels with same mechanics (corrected masks)

In [1]:
from skimage import measure
from PIL import Image
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
paths = ['241218_143216_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_40k_60k/241218_143532_Plate 1/#24B2570319_ZProj',
         '241218_112544_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_peg_10k/241218_134630_Plate 2/#24B2570319_ZProj',
         '241221_122140_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d7_peg_10k/241221_122140_Plate 1/#24B2570319_ZProj',
         '241221_133248_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d7_40k_60k/241221_133248_Plate 1/#24B2570319_ZProj',
         '241229_114731_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d14_all_samples/241229_114731_Plate 1/#24B2570319_ZProj'
]
paths = ['/mnt/z/James/norbornene homopolymerization/same_mechanics/' + p for p in paths]
files = [glob.glob(path+'/*TRITC 556,600]_001_masks_corrected.png') for path in paths]
files = np.array([f for sublist in files for f in sublist])
len(files)

238

In [3]:
files[0]

np.str_('/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_143216_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_40k_60k/241218_143532_Plate 1/#24B2570319_ZProj/C4ROI5_-2_2_1_ZProj[Confocal TRITC 556,600]_001_masks_corrected.png')

In [4]:
'_'.join(files[0].split('_')[:16])

'/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_143216_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_40k_60k/241218_143532'

In [5]:
'_'.join(files[0].split('_')[:18])

'/mnt/z/James/norbornene homopolymerization/same_mechanics/241218_143216_jg_ve_norha_mw_encapsulation_stiff_match_dapi_phal_d3_40k_60k/241218_143532_Plate 1/#24B2570319_ZProj/C4ROI5'

In [7]:
mhc_list = list()

In [8]:
for i,f in enumerate(files):
    #read in masks for calculations
    fov = '_'.join(f.split('_')[:18])
    well, roi = fov.split('/')[-1].split('ROI')
 
    cy5_masks = measure.label(np.array(Image.open(f))[:,:,0])-1
    #cytoskeleton morphometrics
    cy5_props = pd.DataFrame(measure.regionprops_table(cy5_masks*1, properties=['axis_major_length','axis_minor_length','perimeter',
                                                                              'area','eccentricity']))
    cy5_props['labels'] = np.arange(1,len(cy5_props)+1)
    cy5_props['aspect'] = cy5_props['axis_major_length']/cy5_props['axis_minor_length']
    cy5_props['form_factor'] = 4*(np.pi)*cy5_props['area']/(cy5_props['perimeter']**2)

    cy5_props['file'] = f
    cy5_props['fov'] = fov
    cy5_props['well'] = well
    cy5_props['roi'] = roi
    mhc_list.append(cy5_props)
    
    if i % 10 == 0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [9]:
mhc_list = pd.concat(mhc_list)

In [10]:
mhc_list.to_csv('/mnt/z/James/norbornene homopolymerization/same_mechanics/cyto_metrics_no_filtering.csv')